In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install sentence_transformers

# 추천시스템- TF-IDF
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel              # 다수 문장에 대한 유사도 계산

# 테스트 데이터
file_path = '/content/drive/MyDrive/NLP/movies_metadata.csv'
df = pd.read_csv(file_path)

# 데이터 전처리
null_overview = df.overview.isnull().sum()                      # 결측치 확인
null_title = df.title.isnull().sum()

df.dropna(subset = ['overview', 'title'], inplace = True)       # 결측치 제거
df.reset_index(drop = True, inplace=True)                       # 인덱스 재설정

data = df.iloc[:10000, :]                                       # 일부 데이터 활용

tfidf = TfidfVectorizer(stop_words= 'english')                  # 불용어 제거
tfidf.fit(data.overview)                                        # overview 컬럼에 대한 단어 사전 생성
tfidf_matrix = tfidf.transform(data.overview)                   # overview 컬럼의 각 문장을 행렬로 변환
similarity = linear_kernel(tfidf_matrix, tfidf_matrix)          # overview에 대한 유사도 구하기
title_index = pd.Series(data= data.index, index = data.title)   # 인덱스와 제목 위치 교환

# 영화 추천함수
def recommend(title, similarity):
    idx = title_index[title]                                                   # 선택한 영화의 제목에서 유사한 영화의 인덱스 구하기
    sim_score = [(idx, score) for idx, score in enumerate(similarity[idx])]    # 모든 영화에 대해 해당 영화와의 유사도를 구함
    sim_score = sorted(sim_score, key=lambda x: -x[1])                         # 유사도에 따른 영화 정렬

    movie_index = []                                                           # 유사도가 가장 큰 영화 인덱스 추출
    for data in sim_score:
        movie_index.append(data[0])
    movie_title = df.title[movie_index]                                        # 유사도가 가장 큰 10개 추천의 제목

    return movie_title

recommend_movie = recommend('Toy Story', similarity)[:10]
print(recommend_movie)

